In [1]:
import tensorflow as tf
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow import keras
from tensorflow.keras.layers import Activation, Dense, Flatten,Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pandas as pd
import itertools
import shutil
import random
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

In [2]:
Train= "/content/drive/MyDrive/3classnew/NEw Data"
Test="/content/drive/MyDrive/3classnew/NEw Data"

In [3]:
import keras
def load_images(directory):
    images = []
    labels = []
    for idx, label in enumerate(uniq_labels):
        for file in os.listdir(directory + "/" + label):
            filepath = directory + "/" + label + "/" + file
            image = cv2.resize(cv2.imread(filepath,cv2.IMREAD_COLOR), (224, 224))
            image = cv2.cvtColor(image , cv2.COLOR_RGB2BGR)
            images.append(image)
            labels.append(idx)
    images = np.array(images)
    labels = np.array(labels)
    return(images, labels)

uniq_labels = sorted(os.listdir(Train))
image, labels = load_images(directory = Train)
if uniq_labels == sorted(os.listdir(Test)):
  X_new, y_new = load_images(directory = Test)

In [4]:
X_train, X_eval, y_train, y_eval = train_test_split(image,labels, test_size =0.2, stratify = labels)

In [5]:
X_train = X_train.astype('float32')/255.0
X_eval= X_eval.astype('float32')/255.0
X_new = X_new.astype('float32')/255.0

In [6]:
y_train = to_categorical(y_train)
y_eval = to_categorical(y_eval)
y_new = to_categorical(y_new)

In [7]:
print("The size of training dataset is: ", X_train.shape)
print("The size of validation dataset is: ", X_eval.shape)

The size of training dataset is:  (168, 224, 224, 3)
The size of validation dataset is:  (42, 224, 224, 3)


In [8]:
train_datagen = ImageDataGenerator(rotation_range=10,  
    width_shift_range=0.1,  
    height_shift_range=0.2, 
    zoom_range=0.10, 
     shear_range=0.1,
    horizontal_flip=False,
    fill_mode="nearest")

In [9]:
Train_data=train_datagen.flow(X_train,y_train,batch_size=27,shuffle=True)
Evalu_data=train_datagen.flow(X_eval,y_eval,batch_size=27,shuffle=True)
New_data= train_datagen.flow(X_new,y_new,batch_size=32,shuffle=True)

In [10]:
VGG16_modelN = tf.keras.models.load_model('/content/drive/MyDrive/3classnew/FinVGG16_3c.h5')

In [11]:
for layer in VGG16_modelN.layers:
    if "BatchNormalization" in layer.__class__.__name__:
        layer.trainable = True 

In [12]:
VGG16_modelN.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.01)

In [14]:
history = VGG16_modelN.fit(Train_data,epochs=10,batch_size= 1,validation_data=Evalu_data,callbacks=[
                        tf.keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            patience=5,
                            restore_best_weights=True),

                        learning_rate_reduction
                    ])

Epoch 1/10
7/7 [==============================] - 21s 2s/step - loss: 2.4483 - accuracy: 0.6012 - val_loss: 1.0075 - val_accuracy: 0.7143 - lr: 1.0000e-05
Epoch 2/10
7/7 [==============================] - 3s 546ms/step - loss: 1.1156 - accuracy: 0.7262 - val_loss: 0.6056 - val_accuracy: 0.7143 - lr: 1.0000e-05
Epoch 3/10
7/7 [==============================] - 3s 471ms/step - loss: 0.6291 - accuracy: 0.7619 - val_loss: 0.4122 - val_accuracy: 0.8333 - lr: 1.0000e-05
Epoch 4/10
7/7 [==============================] - 4s 492ms/step - loss: 0.4448 - accuracy: 0.8452 - val_loss: 0.3518 - val_accuracy: 0.8810 - lr: 1.0000e-05
Epoch 5/10
7/7 [==============================] - 3s 546ms/step - loss: 0.2679 - accuracy: 0.9167 - val_loss: 0.0628 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 6/10
7/7 [==============================] - 4s 487ms/step - loss: 0.2104 - accuracy: 0.9286 - val_loss: 0.1387 - val_accuracy: 0.9524 - lr: 1.0000e-05
Epoch 7/10
7/7 [==============================] - 4s 489ms/s

In [15]:
 VGG16_modelN.save('/content/drive/MyDrive/3classnew/ VGG16_modelN.h5')    # after fine tuning trained on other data set

**Testing the Retrained Model**

In [22]:
score = VGG16_modelN.evaluate(New_data, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = VGG16_modelN.evaluate(X_new,y_new, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

Accuracy for test images: 98.571 %
Accuracy for evaluation images: 100.0 %


**Inception V3**

In [23]:
Inception_modelN = tf.keras.models.load_model('/content/drive/MyDrive/3classnew/FinIncep_3c.h5')

In [24]:
for layer in Inception_modelN.layers:
    if "BatchNormalization" in layer.__class__.__name__:
        layer.trainable = True 

In [49]:
Inception_modelN.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [50]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.01)

In [53]:
history4 = Inception_modelN.fit(Train_data,epochs=10,batch_size= 1,validation_data=Evalu_data,callbacks=[
                        tf.keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            patience=5,
                            restore_best_weights=True),

                        learning_rate_reduction
                    ])

Epoch 1/10
7/7 [==============================] - 64s 1s/step - loss: 2.4276 - accuracy: 0.6488 - val_loss: 3.3350 - val_accuracy: 0.3810 - lr: 1.0000e-04
Epoch 2/10
7/7 [==============================] - 3s 469ms/step - loss: 0.7529 - accuracy: 0.8214 - val_loss: 1.3463 - val_accuracy: 0.6667 - lr: 1.0000e-04
Epoch 3/10
7/7 [==============================] - 3s 468ms/step - loss: 0.2195 - accuracy: 0.9464 - val_loss: 0.4636 - val_accuracy: 0.8810 - lr: 1.0000e-04
Epoch 4/10
7/7 [==============================] - 3s 477ms/step - loss: 0.1145 - accuracy: 0.9643 - val_loss: 0.4075 - val_accuracy: 0.8810 - lr: 1.0000e-04
Epoch 5/10
7/7 [==============================] - 3s 479ms/step - loss: 0.0766 - accuracy: 0.9762 - val_loss: 1.3968 - val_accuracy: 0.7619 - lr: 1.0000e-04
Epoch 6/10
7/7 [==============================] - 3s 534ms/step - loss: 0.0796 - accuracy: 0.9821 - val_loss: 0.2754 - val_accuracy: 0.9048 - lr: 1.0000e-04
Epoch 7/10
7/7 [==============================] - 3s 419ms/s

In [54]:
Inception_modelN.save('/content/drive/MyDrive/3classnew/Inception_modelN.h5')    # after fine tuning trained on other data set

In [11]:
Inception_modelN = tf.keras.models.load_model('/content/drive/MyDrive/3classnew/FinIncep_3c.h5')

**Testing the Retrained Model**

In [14]:
Inception_modelNn = tf.keras.models.load_model('/content/drive/MyDrive/3classnew/Inception_modelN.h5')

Current model after tranning on other data set tested 

In [16]:
score = Inception_modelNn.evaluate(New_data, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = Inception_modelNn.evaluate(Evalu_data, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

Accuracy for test images: 93.81 %
Accuracy for evaluation images: 95.238 %


**ResNet50**

In [10]:
ResNet50_modelN = tf.keras.models.load_model('/content/drive/MyDrive/3classnew/Resnet50New1_3c.h5')

In [21]:
for layer in ResNet50_modelN.layers:
    if "BatchNormalization" in layer.__class__.__name__:
        layer.trainable = True

In [22]:
ResNet50_modelN.compile(optimizer=Adam(learning_rate=0.000001), loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.01)

In [24]:
history14 = ResNet50_modelN.fit(Train_data,epochs=30,batch_size= 1,validation_data=Evalu_data,callbacks=[
                        tf.keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            patience=5,
                            restore_best_weights=True),

                        learning_rate_reduction
                    ])

Epoch 1/30
7/7 [==============================] - 9s 634ms/step - loss: 3.9580 - accuracy: 0.7262 - val_loss: 9.9364 - val_accuracy: 0.3333 - lr: 1.0000e-06
Epoch 2/30
7/7 [==============================] - 3s 445ms/step - loss: 4.4929 - accuracy: 0.7024 - val_loss: 3.3125 - val_accuracy: 0.3333 - lr: 1.0000e-06
Epoch 3/30
7/7 [==============================] - 3s 427ms/step - loss: 5.9868 - accuracy: 0.6964 - val_loss: 1.8148 - val_accuracy: 0.3333 - lr: 1.0000e-06
Epoch 4/30
7/7 [==============================] - 3s 429ms/step - loss: 2.7868 - accuracy: 0.7440 - val_loss: 2.1088 - val_accuracy: 0.3571 - lr: 1.0000e-06
Epoch 5/30
7/7 [==============================] - 3s 422ms/step - loss: 3.0180 - accuracy: 0.7857 - val_loss: 5.0139 - val_accuracy: 0.3095 - lr: 1.0000e-06
Epoch 6/30
7/7 [==============================] - 3s 427ms/step - loss: 2.7297 - accuracy: 0.6786 - val_loss: 1.8120 - val_accuracy: 0.3333 - lr: 1.0000e-06
Epoch 7/30
7/7 [==============================] - 3s 488ms

In [26]:
ResNet50_modelN.save('/content/drive/MyDrive/3classnew/ResNet50_modelN.h5')    # after fine tuning trained on other data set

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Current model after tranning on other data set tested

In [13]:
score = ResNet50_modelN.evaluate(New_data, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = ResNet50_modelN.evaluate(Evalu_data, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

Accuracy for test images: 42.857 %
Accuracy for evaluation images: 33.333 %


In [25]:
score = ResNet50_modelN.evaluate(New_data, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = ResNet50_modelN.evaluate(Evalu_data, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')


Accuracy for test images: 38.571 %
Accuracy for evaluation images: 35.714 %


**MobileNetV2**

In [15]:
MobileNetV2_modelN = tf.keras.models.load_model('/content/drive/MyDrive/3classnew/MobileNetV2_3c.h5')

In [ ]:
for layer in ResNet50_modelN.layers:
    if "BatchNormalization" in layer.__class__.__name__:
        layer.trainable = True

In [16]:
MobileNetV2_modelN.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.01)

In [18]:
history6 = MobileNetV2_modelN.fit(Train_data,epochs=30,batch_size= 1,validation_data=Evalu_data,callbacks=[
                        tf.keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            patience=5,
                            restore_best_weights=True),

                        learning_rate_reduction
                    ])

Epoch 1/30
7/7 [==============================] - 8s 652ms/step - loss: 0.9960 - accuracy: 0.6250 - val_loss: 0.5708 - val_accuracy: 0.8333 - lr: 1.0000e-05
Epoch 2/30
7/7 [==============================] - 3s 391ms/step - loss: 0.5636 - accuracy: 0.7738 - val_loss: 0.4997 - val_accuracy: 0.8333 - lr: 1.0000e-05
Epoch 3/30
7/7 [==============================] - 3s 392ms/step - loss: 0.3605 - accuracy: 0.8690 - val_loss: 0.2415 - val_accuracy: 0.9048 - lr: 1.0000e-05
Epoch 4/30
7/7 [==============================] - 3s 441ms/step - loss: 0.1358 - accuracy: 0.9464 - val_loss: 0.0520 - val_accuracy: 1.0000 - lr: 1.0000e-05
Epoch 5/30
7/7 [==============================] - 3s 430ms/step - loss: 0.1434 - accuracy: 0.9524 - val_loss: 0.1249 - val_accuracy: 0.9524 - lr: 1.0000e-05
Epoch 6/30
7/7 [==============================] - 3s 398ms/step - loss: 0.1979 - accuracy: 0.9167 - val_loss: 0.0453 - val_accuracy: 0.9762 - lr: 1.0000e-05
Epoch 7/30
7/7 [==============================] - 3s 384ms

Current model after tranning on other data set tested

In [19]:
score = MobileNetV2_modelN.evaluate(New_data, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = MobileNetV2_modelN.evaluate(Evalu_data, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

Accuracy for test images: 100.0 %
Accuracy for evaluation images: 100.0 %


In [20]:
MobileNetV2_modelN.save('/content/drive/MyDrive/3classnew/MobileNetV2_modelN.h5')    # after fine tuning trained on other data set

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
